In [1]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import cv2

In [3]:
# load the BP_utils.py script
os.chdir('/Users/afinneg2/BP_identification')
cwd=os.getcwd()
print cwd
print os.listdir(cwd)
os.chdir('scripts')
import BP_utils
#reload(BP_utils)
os.chdir(cwd)

/Users/afinneg2/BP_identification
['.ipynb_checkpoints', 'analysis_notebooks', 'commands.bash', 'data', 'documents', 'imgs_id_test.npy', 'imgs_mask_train.npy', 'imgs_test.npy', 'imgs_train.npy', 'mnist_data', 'raw', 'scratch', 'scripts']


In [ ]:
# load training data
train_imgs= np.load('imgs_train.npy')
print 'shape train_imgs {}'.format(np.shape(train_imgs))
train_masks = np.load('imgs_mask_train.npy')
print 'shape train_mask {}'.format(np.shape(train_masks))

In [ ]:
print "size of train_imgs is"
print "{} GB".format(sys.getsizeof(train_imgs)/(10.0)**9)

# Obtain the distribution of mask areas for the training examples.  (in preparation for creating a baseline classifier)

In [ ]:
# load the fulll set of training masks
train_masks = np.load('imgs_mask_train.npy')
print 'shape train_mask {}'.format(np.shape(train_masks))

In [ ]:
true_mask_areas = np.zeros(np.shape(train_masks)[0], dtype=float)
for im_index in xrange(0, np.shape(train_masks)[0]):
    train_mask = train_masks[im_index,0]
    true_mask_areas[im_index] =np.count_nonzero(train_mask >0

In [ ]:
%matplotlib inline

print "number of training examles {}".format(len(true_mask_areas) )
print "number of training examples positive for BP {}".format(
                            np.count_nonzero(true_mask_areas) 
                                                        )
print "fraction of images with BP {}".format(
                        np.count_nonzero(true_mask_areas)/np.float(
                               len(true_mask_areas) )  )

### get the mean mask area for training examples positive for
### BP

BP_pos_mask = true_mask_areas > 0.0
mean_pos_mask_area =np.mean(true_mask_areas[BP_pos_mask])



print "mean mask area over masks postiive for BP \n{:.4}".format(
                                                    mean_pos_mask_area)

print "fraction mask area for masks positive for BP\n{:.4f}".format(
                                    mean_pos_mask_area/(420.0*580.0))


fig=plt.figure()
ax=fig.add_subplot(1,1,1)
ax.hist(true_mask_areas)
ax.set_xlabel('mask area')
ax.set_ylabel('count')

# Get a baseline for calling BP regions 

### Baseline prediction accuracy is generated as follows
+ select 41 % of training images as positive for BP
+ For each training image selected as positive call BP as ciricular retion with radius corresponding to mean ground truth mask area and center uniformly distributed in images such that entier circular mask is with image
+ report the mean dice coefficient over the data set (this mean dice coefficient is used to assess the performance of classifiers)

In [ ]:
nb_train_imgs =np.shape(train_masks)[0]
call_area = 7126


radius = np.sqrt(call_area /np.pi)
called_pos_indices =np.random.choice(np.arange(0,nb_train_imgs),
                                     size=np.int(np.floor(0.412*nb_train_imgs)),
                                     replace=False)

dice_coeffs=np.ones(shape=(nb_train_imgs,), dtype=float)
for img_index in xrange(0,nb_train_imgs):
    train_mask =train_masks[img_index,0]    
    if img_index in called_pos_indices:
        center = (np.random.uniform(0+radius, 580 - radius),
                  np.random.uniform(0+radius, 420 - radius ))
        called_mask = BP_utils.mask_from_circle_params(center, radius)
        called_mask=called_mask.astype(float)
    else:
        called_mask=np.zeros(shape=(420,580), dtype=float)
        
    y_true  = train_mask>0
    y_true=y_true.astype(float)
    y_pred=called_mask.astype(float)
    dice_coeffs[img_index]=BP_utils.dice_coef(y_true, y_pred)

In [ ]:
print np.mean(dice_coeffs

In [ ]:
plt.hist(dice_coeffs)

In [ ]:
print radius